In [2]:
!pip install langchain langchain-community chromadb ollama sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 11.4 MB/s eta 0:0

In [3]:
!pip install pypdf langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [4]:
!pip install -U langchain-ollama

In [5]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ··········


# Document Loader

In [6]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("1 Basic Chess Instructions.pdf")
pages = []
async for page in loader.alazy_load():
  pages.append(page)

In [7]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'producer': 'GNU Ghostscript 7.05', 'creator': 'dvips(k) 5.92b Copyright 2002 Radical Eye Software', 'creationdate': '', 'title': 'chessrules3.dvi', 'source': '1 Basic Chess Instructions.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}

BASIC RULES OF CHESS
Introduction
Chess is a game of strategy believed to have been invented
more then 1500 years ago in India. It is a game for two play-
ers, one with the light pieces and one with the dark pieces.
The chessboard is eight squares long by eight squares wide.
When sitting across the board from another player, the
lighter color square goes on each player's right hand side
(light on right) and the white queen is placed on a white
square and the black queen on the black square (queen on
her own color). The starting position of the pieces is shown
in the following chessboard:
8
     
7     
6
5
4
3
2     
1    	 
a b c d e f g h
The player with the white pieces moves rst and the players
then take turns moving. One pi

# Text Splitting

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(pages)

# Embeddings

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Storing Embeddings in a Vector Store

In [10]:
from langchain_community.vectorstores import Chroma

vectordb = Chroma.from_documents(documents = chunks, embedding = embeddings_model, persist_directory="vector_db")
vectordb.persist()

/tmp/ipython-input-1663094989.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


# Setting up a LLM

In [11]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

# Creating a RetrievalQA chain

In [12]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": vectordb.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Let's start asking questions from our chatbot

In [13]:
qa_chain.invoke("How does a rook moves?")

"The Rook can move any number of squares in one direction, either vertically or horizontally. Its path must not be blocked by other pieces. For example, it can capture an opponent's piece if it's in its path, but cannot move through its own pieces."

In [15]:
# i'll try asking something it doesn't know
qa_chain.invoke("Which studio is behind development of Death Stranding 2?")

"I don't know the answer. The provided context describes the rules of chess and does not contain any information about the development studio of Death Stranding 2."

In [ ]:
# Works as intended!! Less Gooo